<H2>Create an I girder model in shell elements</H2>

Create a girder class to hold the section dimensions for each segment
We'll create points in the LUSAS model for each section and then use the points to create surfaces joining them.

In [ ]:
class Girder:
    def __init__(self, tfb:float, tfthk:float, dw:float, tw:float, bfb:float, bfthk:float):
        self.top_flange_breadth = tfb
        self.top_flange_thk = tfthk
        self.web_depth = dw
        self.web_thk = tw
        self.bottom_flange_breadth = bfb
        self.bottom_flange_thk = bfb
        # Wé'll save the points in these lists
        self.bf_points = []
        self.tf_points = []


Define a list of segment lengths so the girder can have changes in sections

In [4]:
segments_lengths = [6000, 5000, 6000]

Create a list of section definitions, we'll need one per segment

In [ ]:
sections = []
sections.append(Girder(500, 50, 1500, 15, 600, 60))
sections.append(Girder(500, 50, 1500, 15, 600, 60))
sections.append(Girder(500, 50, 1500, 15, 600, 60))

# Check we have a section for each segment
assert(len(segments_lengths) == len(sections))

Connect to LUSAS Modeller

In [3]:
import win32com.client as win32
lusas = win32.gencache.EnsureDispatch("Lusas.Modeller.21.0")
db = lusas.database()

Create a helper function to create a point in the model

In [ ]:
def create_point_in_modeller(x:float, y:float, z:float) -> 'IFPoint':
    return win32.CastTo(db.createPoint(x, y, z))

Create all the points for each section

In [ ]:
s:Girder
for i, s in enumerate(sections):
    # Determine the longitudinal position (x coord) for each section
    x = 0 if i == 0 else segments_lengths[i-1]
    # Create the bottom flange points
    s.bf_points.append(create_point_in_modeller(x, +s.bottom_flange_breadth/2, 0.0))
    s.bf_points.append(create_point_in_modeller(x, 0.0, 0.0))
    s.bf_points.append(create_point_in_modeller(x, -s.bottom_flange_breadth/2, 0.0))
    # Create the top flange points
    s.tf_points.append(create_point_in_modeller(x, +s.top_flange_breadth/2, 0.0))
    s.tf_points.append(create_point_in_modeller(x, 0.0, 0.0))
    s.tf_points.append(create_point_in_modeller(x, -s.top_flange_breadth/2, 0.0))

Now join up the segment points. First create helper function to create surfaces and assign a geometric thickness attribute

In [ ]:
def create_surface_in_lusas(pnts: list, thk:float, ecc:float, group:str):

    geometryData = lusas.geometryData()
    geometryData.setAllDefaults()
    geometryData.setCreateMethod("coons")
    geometryData.setLowerOrderGeometryType("points")

    obs = lusas.newObjectSet()
    for p in pnts:
        obs.add(p)

    objs = obs.createSurface(geometryData).getObjects("Surface")
    grp = db.getGroupByName(group)
    grp.add(objs, "Surfaces")

    name = f"{group} ({thk:.1f}mm)"

    db.createGeometricSurface(name).setSurface(thk, ecc).assignTo(objs)


Create groups to contain the surfaces

In [ ]:
db.createEmptyGroup("Top Flange")
db.createEmptyGroup("Web")
db.createEmptyGroup("Bottom Flange")

In [ ]:
for i, s in range(0, len(sections)):

    s1:Girder = sections[i]
    s2:Girder = sections[i+1]

    create_surface_in_lusas([s1.bf_points[0], s2.bf_points[0], s2.bf_points[1], s1.bf_points[1] ], s1.bottom_flange_thk, 0.0, "Bottom Flange")
    create_surface_in_lusas([s1.bf_points[1], s2.bf_points[1], s2.bf_points[2], s1.bf_points[2] ], s1.bottom_flange_thk, 0.0, "Bottom Flange")
    create_surface_in_lusas([s1.bf_points[1], s1.tf_points[1], s2.tf_points[1], s2.bf_points[1] ], s1.web_thk, 0.0, "Web")
    create_surface_in_lusas([s1.tf_points[0], s2.tf_points[0], s2.tf_points[1], s1.tf_points[1] ], s1.top_flange_thk, 0.0, "Top Flange")
    create_surface_in_lusas([s1.tf_points[1], s2.tf_points[1], s2.tf_points[2], s1.tf_points[2] ], s1.top_flange_thk, 0.0, "Top Flange")